## Urbanpy model for Para for car transportation

Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

# Only needed when git cloning the urbanpy repo
# import sys
# sys.path.append('..')

import urbanpy as up
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import plotly.express as px 
import osmnx as ox

from tqdm.notebook import tqdm
tqdm.pandas()

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

Download maps

In [ ]:
state = 'Pará, Brazil'
para = up.download.nominatim_osm(state)

Plot map

In [1]:
para.plot()
plt.title(state)

NameError: name 'para' is not defined

## Divide Para into hexagons for spatial analysis

In [ ]:
h3_resolution = 7

hex_para = up.geom.gen_hexagons(h3_resolution, city=para)

In [ ]:
hex_para.plot(edgecolor='none')
plt.title(state + ". Cells H3 with resolution " + str(h3_resolution))

## Download population data

In [ ]:
full_pop_brazil_northeast = up.download.hdx_dataset(resource="c17003d1-47f4-4ec5-8229-2f77aeb114be/resource/957218ee-c740-44c0-88e5-7faeef813a0c/download/population_bra_northeast_2018-10-01.csv.zip")
full_pop_brazil_northwest = up.download.hdx_dataset(resource="c17003d1-47f4-4ec5-8229-2f77aeb114be/resource/1e1f271b-1055-4365-b391-f6fdf3093fe2/download/population_bra_northwest_2018-10-01.csv.zip")

In [ ]:
full_pop_brazil_north = pd.concat([full_pop_brazil_northeast, full_pop_brazil_northwest])

Filter data to just population in Para

In [ ]:
pop_para = up.geom.filter_population(full_pop_brazil_north, para)
pop_para.head()

Multiply population by .07 to approximate children at school age

In [ ]:
## multiply every row in pop_para by 0.07
pop_para['population_2020'] = pop_para['population_2020'].apply(lambda x: x*0.07)
pop_para.head()

## Calculate population per hexagon

In [ ]:
hex_para = up.geom.merge_shape_hex(
    hexs = hex_para, 
    shape = pop_para, 
    agg={'population_2020': 'sum'}
)

## Import school data

In [ ]:
es_updated = pd.read_csv('data/escuelas.csv', skiprows=1)

Rename columns

In [ ]:
es_updated = es_updated.rename(columns={'LAT_FINAL': 'lat'})
es_updated = es_updated.rename(columns={'LONG_FINAL': 'lon'})

We also need a new column in the dataset called geometry that is a shapely point object

In [ ]:
es_updated['geometry'] = es_updated.apply(lambda row: 'POINT (' + str(row['lon']) + ' ' + str(row['lat']) + ')', axis=1)

## Subset school data 

In [ ]:
# remove rows from `es_updated` where 'Etapas e Modalidade de Ensino Oferecidas' is 'Ensino Médio, Educação Profissional' or 'Ensino Médio' or 'Educação de Jovens Adultos' or 'Ensino Médio, Educação de Jovens Adultos' or 'Educação Profissional' or 'Ensino Médio, Educação Profissional, Educação de Jovens Adultos' or '   'Educação Profissional, Educação de Jovens Adultos'
es_updated = es_updated[~es_updated['Etapas e Modalidade de Ensino Oferecidas'].isin(['Ensino Médio, Educação Profissional', 'Ensino Médio', 'Educação de Jovens Adultos', 'Ensino Médio, Educação de Jovens Adultos', 'Educação Profissional', 'Ensino Médio, Educação Profissional, Educação de Jovens Adultos', 'Educação Profissional, Educação de Jovens Adultos'])]

And then subset to just Para

In [ ]:
es_para_updated = es_updated[es_updated['UF'] == 'PA']
es_para_updated.head(3)

## Evaluate accessibility

In [ ]:
hex_para['lat'] = hex_para.geometry.centroid.y
hex_para['lon'] = hex_para.geometry.centroid.x

In [ ]:
dist_up, ind_up = up.utils.nn_search(
    tree_features=es_para_updated[['lat', 'lon']].values, # These are the schools
    query_features=hex_para[['lat', 'lon']].values, # Values are the centroids of each hexagon
    metric='haversine'
)

In [ ]:
hex_para['closest_school'] = ind_up

## Download data needed for Para

!cd ~/data/osrm && wget https://download.geofabrik.de/south-america/brazil/norte-latest.osm.pbf

start OSRM server

In [ ]:
up.routing.start_osrm_server('norte', 'south-america_brazil', 'car')

Need the below line of code to convert the data to a format that OSRM can read

In [ ]:
from shapely.wkt import loads
es_para_updated['geometry'] = es_para_updated['geometry'].apply(lambda x: loads(x))

Then do our distance and time calculations

In [ ]:
distance_duration_para_car = hex_para.apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid, 
        destination = es_para_updated.iloc[row['closest_school']]['geometry']
    ),
    result_type='expand',
    axis=1,
)

## Inspect results

- `0` corresponds to the distance
- `1` corresponds to the duration of the trip by foot

Our next step is to bring these data into the `hex_para` variable

In [ ]:
distance_duration_para_car.head()

In [ ]:
hex_para['distance_to_school_km'] =  distance_duration_para[0] / 1000 # metros a km
hex_para['duration_to_school_min'] = distance_duration_para[1] / 60 # segundos a minutos

In [ ]:
hex_para['distance_to_school_km'] =  distance_duration_para_car[0] / 1000 # metros a km
hex_para['duration_to_school_min'] = distance_duration_para_car[1] / 60 # segundos a minutos

In [ ]:
# Once we have finished with the OSRM server we stop it
up.routing.stop_osrm_server('norte', 'north-america_brazil', 'car')

In [ ]:
hex_plot_para = hex_para.query("population_2020 > 0").reset_index(drop=True)
# Reset index is needed to avoid an error with plotly choropleth_map

In [ ]:
fig = up.plotting.choropleth_map(hex_plot_para, 'duration_to_school_min', 
                                title=  state + '. Estimated travel times to school by car',
                                zoom = 5, 
                                color_continuous_scale="Plasma_r",
                                opacity=0.6, 
                                labels={'duration_to_school_min':'Duration (min)'}
                                )

fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
)
fig.update_traces(marker_line_width=0.0)
fig.show()